## Sequential Modeling

Romen Samuel Rodis Wabina <br>
Student, PhD Data Science in Healthcare and Clinical Informatics <br>
Clinical Epidemiology and Biostatistics, Faculty of Medicine (Ramathibodi Hospital) <br>
Mahidol University <br>

In [13]:
import numpy as np 
import tensorflow as tf
from tensorflow import keras 
from keras import layers 

max_features = 20000
maxlen = 200

In [14]:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [17]:
(X_train, y_train), (X_val, y_val) = keras.datasets.imdb.load_data(num_words = max_features)

X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen = maxlen)
X_val   = keras.preprocessing.sequence.pad_sequences(X_val,   maxlen = maxlen)

In [20]:
inputs = keras.Input(shape = (None, ), dtype = 'int32')

x = layers.Embedding(max_features, 128)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)

outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_5 (Embedding)     (None, None, 128)         2560000   
                                                                 
 bidirectional_8 (Bidirectio  (None, None, 128)        98816     
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,757,761
Trainable params: 2,757,761
Non-tra

In [22]:
model.compile('adam', 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 128, epochs = 2, validation_data = (X_val, y_val))

Epoch 1/2
 11/196 [>.............................] - ETA: 12:04 - loss: 0.2598 - accuracy: 0.8991